# Exploratory data analysis

In this notebook, we will be conducting some exploratory data analysis for the `built-environment` project.
I don't promise to be comprehensive. I'm just trying to get *something* done, to find a starting point.

In [ ]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

## Staring at tables from FARS data

Let's start with `accident.csv` from the FARS 2022 National data.
At first, I kept getting a `UnicodeDecodeError` reading `accident.csv` with `pandas`.
Apparently, the error was caused by the format, as described in this [StackOverflow thread](https://archive.today/2025.03.05-154624/https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas).

To fix the error, I first made a copy `accident-copy.csv` so that I didn't alter the raw data.
I opened `accident-copy.csv` in VS Code, clicked the UTF-8 label in the bottom-right corner, then clicked "Save with encoding" and selected UTF-8.
I was then able to properly read the CSV.

In [ ]:
accident_df = pd.read_csv("../data/processed/FARS/2022/accident-copy.csv")
accident_df

In [ ]:
accident_df.describe()

There are way too many CSV files for me to investigate in such ashort time, so I'll focus on a few important ones.
By skimming the [FARS Analytical User Manual](https://web.archive.org/web/20250226140538/https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/813556), I was able to determine a few files that seem worthy of attention.
- `accident.csv` describes the crash characteristics and environmental conditions for each crash.
- `person.csv` is important because it describes injury severity.
- `weather.csv` describes atmospheric conditions.
- `vision.csv` describes visibility conditions.
- I don't think I'll look at `drugs.csv` right now, but it might be worth controlling for toxicology.

Let's get some more data. 
- If I run into a `UnicodeDecodeError`, then I will perform the same fix as before.
- Otherwise, I will proceed normally.

In [ ]:
person_df = pd.read_csv("../data/raw/FARS/2022/FARS2022NationalCSV/person.csv")
weather_df = pd.read_csv("../data/raw/FARS/2022/FARS2022NationalCSV/weather.csv")
vision_df = pd.read_csv("../data/raw/FARS/2022/FARS2022NationalCSV/vision.csv")
person_df

In [ ]:
person_df.describe()

In [ ]:
weather_df

In [ ]:
weather_df.describe()

In [ ]:
vision_df

In [ ]:
vision_df.describe()

Here are some relevant columns from each CSV.
- `ST_CASE` is the unique ID for each crash. (There are columns with unique identifiers for each vehicle and person involved, but I don't think our project needs this level of detail.)
- Obviously, we're interested in `MONTH`, `DAY`, `HOUR`, and `MINUTE`. (Of course, the year is 2022 for all of this data.)
    - We can also look at `DAY_WEEK` for day-of-the-week. Sunday is 1, Monday is 2, ..., Saturday is 7.
- From `accident.csv`, we have many other relevant columns.
    - We have some columns counting types of people involved in a crash.
        - `PEDS` counts people not in motor vehicles.
        - `PERNOTMVIT` counts non-motorists in transit, e.g. cyclists. 
        - `PERSONS` counts "Person Forms", and `PERMVIT` counts motorists.
    - `VE_TOTAL` counts the number of vehicles involved in a crash.
        - `VE_FORMS` counts vehicles in-transport.
        - `PVH_INVL` counts parked and working vehicles.
    - Global position is recorded with `LATITUDE` and `LONGITUD`.
        - For `LATITUDE`:
            - DD.DDDDDDD = actual decimal degrees
            - 77.7777000 = not reported
            - 88.8888000 = not available
            - 99.9999000 = reported as unknown
        - Similar idea for `LONGITUD`:
            - -DDD.DDDDDDD = actual decimal degrees
            - 777.7777000 = not reported
            - 888.8888000 = not available
            - 999.9999000 = reported as unknown
        - There are also other columns for location, e.g. `COUNTY` and `CITY`. (It's worth caring about `COUNTY` if we're restricting to LA, OC, and SD counties.)
    - `HARM_EV` is a categorical variable describing the first injury- or damage-producing event of the crash.
    - There are some columns that are related to the built environment.
        - `ROUTE` describes route signing, e.g. Interstate, US Highway, State Highway, County Road, Local Street, etc.
        - `RUR_URB` describes land use, e.g. rural vs urban.
        - `FUNC_SYS` describes classification of the segment of the trafficway where the crash occurred, e.g. principal arterial, minor arterial, major collector, minor collector, etc.
        - `RELJCT1` describes whether a crash occurred close to an interchange.
        - `RELJCT2` describes proximity to components, e.g. intersection.
        - `TYP_INT` describes type of intersection.
        - `REL_ROAD` describes relation to trafficway, e.g. roadway, median, shoulder, roadside, etc.
        - `WRK_ZONE` describes whether the first harmful event occurred in a work zone.
    - `LGT_COND` describes the light conditions, e.g. dawn, dusk, etc.
    - `WEATHER` describes atmospheric conditions.
    - There are some columns related to when EMS was called, when EMS arrived, when patients arrived at the hospital, etc. (These columns are relevant as confounding factors for whether or not a crash is fatal. Maybe quick EMS response times saved lives in a really bad crash, vs slow EMS response times caused people to die in a less severe crash.)
    - `FATALS` counts number of fatalities.
- From `person.csv`, the `INJ_SEV` column describes injury severity as categorical data. (Since we will be looking at crash-level data and not vehicle-level or person-level, maybe we should care about the highest injury level seen in the crash, or maybe some kind of average?)
    - 0 = no apparent injury
    - 1 = possible injury
    - 2 = suspected minor injury
    - 3 = suspected serious injury
    - 4 = fatal injury
    - 5 = injured, severity unknown
    - 6 = died prior to crash
    - 9 = unknown/unreported
- `person.csv` also has some columns that could indicate confounding factors, like `REST_USE` (restraint use), `REST_MIS` (restraint misuse), `HELM_USE` (helmet use), `HELM_MIS` (helmet misuse), `DRINKING` (alcohol use), `DRUGS` (toxicology), etc.
- The `WEATHER` column from `weather.csv` obviously describes atmospheric conditions.
    - 1 = clear
    - 2 = rain
    - 3 = sleet or hail
    - 4 = snow
    - 5 = fog, smog, smoke
    - 6 = severe crosswinds
    - 7 = blowing sand, dirt, or soil
    - 8 = other
    - 10 = cloudy
    - 11 = blowing snow
    - 12 = freezing drain or drizzle
    - 98 = not reported
- The `VISION` column from `vision.csv` obviously describes visual obstructions.
    - 0 = no obstruction noted
    - 1 = rain, snow, fog, smoke, sand, dust
    - 2 = reflected glare, bright sunlight, headlights
    - 3 = curve, hill, or other roadway design features
    - 4 = building, billboard, or other structure
    - 5 = trees, crops, vegetation
    - 6 = in-transport motor vehicle
    - 7 = not in-transport motor vehicle
    - 8 = splash or spray of passing vehicle
    - 9 = inadequate defrost or defog system
    - 10 = inadequate vehicle lighting system
    - 11 = obstructing interior to the vehicle
    - 12 = external mirrors
    - 13 = broken or improperly cleaned windshields
    - 14 = obstructing angles on vehicle
    - 95 = no driver present/unknown if driver present
    - 97 = vision obscured, no details
    - 98 = other visual obstruction
    - 99 = reported as unknown

## A little bit of data processing for FARS data

I'd like to process the data a bit.
- I want to focus on crashes occurring only in LA, OC, and SD counties.
    - According to the FARS Analytical User Manual, `COUNTY` uses Geographic Location Codes (GLCs) from the General Services Administration (GSA).
    - From the [GSA website](http://archive.today/2025.03.12-083949/https://www.gsa.gov/reference/geographic-locator-codes), I was able to find the relevant state and county codes.
        - California = 06
        - LA = 037
        - OC = 059
        - SD = 073
- Once I have acquired the unique IDs `ST_CASE` for these counties, I can select relevant columns.
- Next, I'll throw away rows where data is unreported.

In [ ]:
accident_df['COUNTY'].dtypes

In [ ]:
accident_df['COUNTY'].values

In [ ]:
county_codes = [37, 59, 73] # for LA, OC, and SD counties
accident_counties_df = accident_df[(accident_df['STATE'] == 6) & (accident_df['COUNTY'].isin(county_codes))]
accident_counties_df



Let's keep the following columns from `accident.csv`.
- Unique ID: `ST_CASE`
- Time: `MONTH`, `DAY`, `HOUR`, `MINUTE`, `DAY_WEEK`
- Location: `LATITUDE`, `LONGITUD`, `COUNTY`, `CITY`
- Number of people involved: `PERSONS`, `PERMVIT`, `PERNOTMVIT`, `PEDS`
- Number of vehicles involved: `VE_TOTAL`, `VE_FORMS`, `PVH_INVL`
- Number of fatalities: `FATALS`
- First harmful event: `HARM_EV`
- Conditions: `LGT_COND`, `WEATHER`, `WRK_ZONE`
- Built environment: `ROUTE`, `RUR_URB`, `FUNC_SYS`, `RELJCT1`, `RELJCT2`, `TYP_INT`, `REL_ROAD`
- EMS response: `NOT_HOUR`, `NOT_MIN`, `ARR_HOUR`, `ARR_MIN`, `HOSP_HR`, `HOSP_MN`

In [ ]:
relevant_columns = ['ST_CASE',
                    'MONTH', 'DAY', 'HOUR', 'MINUTE', 'DAY_WEEK',
                    'LATITUDE', 'LONGITUD', 'COUNTY', 'CITY',
                    'PERSONS', 'PERMVIT', 'PERNOTMVIT', 'PEDS',
                    'VE_TOTAL', 'VE_FORMS', 'PVH_INVL',
                    'FATALS',
                    'HARM_EV',
                    'LGT_COND', 'WEATHER', 'WRK_ZONE',
                    'ROUTE', 'RUR_URB', 'FUNC_SYS', 'RELJCT1', 'RELJCT2', 'TYP_INT', 'REL_ROAD',
                    'NOT_HOUR', 'NOT_MIN', 'ARR_HOUR', 'ARR_MIN', 'HOSP_HR', 'HOSP_MN']
accident_relevant_df = accident_counties_df[relevant_columns]
accident_relevant_df

Let's get rid of rows with unreported/unknown values.

In [ ]:
accident_relevant_df.dtypes

In [ ]:
# Time
valid_month = [x for x in range(1,13)]
valid_day = [x for x in range(1,32)]
valid_day_of_week = [x for x in range(1,8)]
valid_hour = [x for x in range(24)]
valid_min = [x for x in range(60)]

# Location
invalid_latitude = [77.7777000, # not reported
                    88.8888000, # not available (if state exempt)
                    99.9999000  # reported as unknown
                    ]
invalid_longitude = [777.7777000, # not reported
                     888.8888000, # not available (if state exempt)
                     999.9999000  # reported as unknown
                     ]

# Harmful event
invalid_event = 99 # reported as unknown

# Conditions
#invalid_light = [8, # not reported
#                 9  # reported as unknown
#                 ]
#invalid_weather = [98, # not reported
#                   99  # reported as unknown
#                   ]
valid_light = [x for x in range(1,8)]
valid_weather = [x for x in range(1,13)]

# Built environment
#invalid_route = 9 # unknown
valid_route = [x for x in range(1,9)]
valid_land_use = [1, # rural
                  2  # urban
                  ]
valid_function = [x for x in range(1,8)]
valid_reljct1 = [0, # not within interchange area
                 1  # within interchange area
                 ]
valid_reljct2 = [x for x in range(1,9)]
valid_reljct2.extend([x for x in range(16,21)])
valid_intersection = [x for x in range(1,12)]
valid_road = [x for x in range(1,9)]
valid_road.extend([x for x in range(10,13)])

accident_processed_df = accident_relevant_df[(accident_relevant_df['MONTH'].isin(valid_month))
                                             & (accident_relevant_df['DAY'].isin(valid_day))
                                             & (accident_relevant_df['DAY_WEEK'].isin(valid_day_of_week))
                                             & (accident_relevant_df['HOUR'].isin(valid_hour))
                                             & (accident_relevant_df['MINUTE'].isin(valid_min))
                                             & (~accident_relevant_df['LATITUDE'].isin(invalid_latitude))
                                             & (~accident_relevant_df['LONGITUD'].isin(invalid_longitude))
                                             & (accident_relevant_df['HARM_EV'] != invalid_event)
                                             & (accident_relevant_df['LGT_COND'].isin(valid_light))
                                             & (accident_relevant_df['WEATHER'].isin(valid_weather))
                                             & (accident_relevant_df['ROUTE'].isin(valid_route))
                                             & (accident_relevant_df['RUR_URB'].isin(valid_land_use))
                                             & (accident_relevant_df['FUNC_SYS'].isin(valid_function))
                                             & (accident_relevant_df['RELJCT1'].isin(valid_reljct1))
                                             & (accident_relevant_df['RELJCT2'].isin(valid_reljct2))
                                             & (accident_relevant_df['TYP_INT'].isin(valid_intersection))
                                             & (accident_relevant_df['REL_ROAD'].isin(valid_road))
                                             ]

accident_processed_df

Okay, at this point, we have a little over 1000 unique crashes.

I'd like to add a few more columns.
- `INJ_SEV` from `person.csv`
- `VISION` from `vision.csv`

(I'll do this later. For now, I'd like to move onto descriptive statistics.)

## Descriptive statistics for the FARS data

Before working on any visualizations, maybe it'd be worth printing out some descriptive statistics.

In [ ]:
accident_processed_df.info

### Time

For time, we might want to know which months, hours, and days of the week have the most crashes.

In [ ]:
accident_processed_df.value_counts('MONTH')

The worst month (October) has about $104/77 \approx 1.35$ times as many crashes as the safest month (December).

In [ ]:
accident_processed_df.value_counts('HOUR')

Wow, the worse time (02:00) has about $81/21 \approx 3.86$ times as many crashes as the safest time (14:00).
It seems like the most dangerous times are at night, and the safest times are in the morning and early afternoon.

In [ ]:
accident_processed_df.value_counts('DAY_WEEK')

The worst day (Sunday) is has $194/103 \approx 1.88%$ times as many crashes as the safest day (Wednesday).
The most dangerous days are on the weekends.

We can also cut up the data a bit.

In [ ]:
accident_processed_df.value_counts(['MONTH', 'HOUR'])

In [ ]:
accident_processed_df.value_counts(['MONTH', 'DAY_WEEK'])

Not sure how helpful that was.

Anyways, let's visualize some of the time data.

In [ ]:
sns.countplot(accident_processed_df, x='MONTH')

In [ ]:
sns.boxplot(accident_processed_df, x='MONTH', y='PERSONS')

In [ ]:
sns.boxplot(accident_processed_df, x='MONTH', y='FATALS')

In [ ]:
sns.boxplot(accident_processed_df, x='MONTH', y='VE_TOTAL')

In [ ]:
#sns.countplot(accident_processed_df, x='MONTH', hue='FATALS')

In [ ]:
#sns.countplot(accident_processed_df, x='MONTH', hue='PERSONS')

In [ ]:
sns.countplot(accident_processed_df, x='HOUR')

In [ ]:
sns.boxplot(accident_processed_df, x='HOUR', y='PERSONS')

In [ ]:
sns.boxplot(accident_processed_df, x='HOUR', y='FATALS')

In [ ]:
sns.boxplot(accident_processed_df, x='HOUR', y='VE_TOTAL')

In [ ]:
#sns.countplot(accident_processed_df, x='HOUR', hue='FATALS')

In [ ]:
#sns.countplot(accident_processed_df, x='MONTH', hue='PERSONS')

In [ ]:
sns.countplot(accident_processed_df, x='DAY_WEEK')

In [ ]:
sns.boxplot(accident_processed_df, x='DAY_WEEK', y='PERSONS')

In [ ]:
sns.boxplot(accident_processed_df, x='DAY_WEEK', y='FATALS')

In [ ]:
sns.boxplot(accident_processed_df, x='DAY_WEEK', y='VE_TOTAL')

In [ ]:
#sns.countplot(accident_processed_df, x='DAY_WEEK', hue='FATALS')

In [ ]:
#sns.countplot(accident_processed_df, x='DAY_WEEK', hue='PERSONS')

### Persons

We might also want to know descriptive statistics for people involved in crashes.

In [ ]:
accident_processed_df[['PERSONS', 'PERMVIT', 'PERNOTMVIT', 'PEDS']].describe()

In [ ]:
accident_processed_df[['PERSONS', 'PERMVIT', 'PERNOTMVIT', 'PEDS']].median()

In [ ]:
sns.displot(accident_processed_df, x='PERSONS')

In [ ]:
sns.displot(accident_processed_df, x='PEDS')

Typically, 2 people are injured per crash, and these people are typically occupants of motor vehicles in transit.

Dang though, looks like there was a crash involving 17 people.

In [ ]:
accident_processed_df['FATALS'].describe()

In [ ]:
accident_processed_df['FATALS'].median()

In [ ]:
sns.displot(accident_processed_df, x='FATALS')

Typically, a crash involves 1 fatality. However, there was a crash with 5 fatalities in 2022.

I'm not sure if I'm reading the data wrong, but something looks off.
- `PERSONS` has a minimum of 0, meaning there has been a crash involving zero people.
- `FATALS` has a minimum of 1, meaning the minimum number of fatalities per crash is 1.
- It's impossible for both of those things to be true simultaneously...

### Vehicles

How about the number of vehicles involved per crash?

In [ ]:
accident_processed_df[['VE_TOTAL', 'VE_FORMS', 'PVH_INVL']].describe()

In [ ]:
accident_processed_df[['VE_TOTAL', 'VE_FORMS', 'PVH_INVL']].median()

In [ ]:
sns.displot(accident_processed_df, x='VE_TOTAL')

In [ ]:
sns.displot(accident_processed_df, x='PVH_INVL')

Looking at these descriptive statistics, we notice a few things.
- Naturally, at least one vehicle is involved per crash. 
- Thankfully, the typical number of vehicles involved per crash is 1.
- Typically, 0 parked vehicles are involved per crash.
- There has been a crash involving as many as 11 vehicles, and a crash involving as many as 5 parked vehicles.

### Harmful events

What harmful events are the most common causes of crashes?

In [ ]:
accident_processed_df.value_counts('HARM_EV')

In [ ]:
sns.countplot(accident_processed_df, x='HARM_EV')

The top 3 causes account for $776/1067 \approx 72.7\%$ of crashes.
1. Pedestrian (8), accounting for $352/1067 \approx 33.0\%$ of crashes
2. Motor vehicle in-transport (12), accounting for $318/1067 \approx 29.8\%$ of crashes
3. Curb (3), accounting for $106/1067 \approx 9.93\%$ of crashes

### Conditions

Now let's look at environmental conditions.

In [ ]:
accident_processed_df.value_counts('LGT_COND')

In [ ]:
sns.countplot(accident_processed_df, x='LGT_COND')

The top 3 light conditions account for $1016/1067 \approx 95.2\%$ of crashes.
1. Dark - Lighted (3), accounting for $474/1067 \approx 44.4\%$ of crashes
2. Daylight (1), accounting for $346/1067 \approx 32.4\%$ of crashes
3. Dark - Not Lighted (2), accounting for $196/1067 \approx 18.4\%$ of crashes

If I were to guess, not too many people drive at Dawn (4) or Dusk (5), and the lighting condition is usually reported, so most people wouldn't be categorized under Dark - Unknown Lighting (6).

In [ ]:
accident_processed_df.value_counts('WEATHER')

In [ ]:
sns.countplot(accident_processed_df, x='WEATHER')

The vast majority ($932/1067 \approx 87.3\%$) of crashes occur under Clear (1) conditions, simply because those are the usual conditions in LA/OC/SD counties.

Cloudy (10) accounts for $102/1067 \approx 9.56\%$ of crashes, while Rain (2) accounts for $26/1067 \approx 2.44\%$.

In [ ]:
accident_processed_df.value_counts('WRK_ZONE')

In [ ]:
sns.countplot(accident_processed_df, x='WRK_ZONE')

Work zones are uncommon, so it makes sense that most accidents occur outside of work zones. We can probably ignore work zones.

### Built environment

Let's examine the built environment.


In [ ]:
accident_processed_df.value_counts('ROUTE')

In [ ]:
sns.countplot(accident_processed_df, x='ROUTE')

The top 3 route signings account for $1025/1067 \approx 96.1\%$ of crashes.
1. Other (8), accounting for $575/1067 \approx 53.9\%$ of crashes
2. Interstate (1), accounting for $245/1067 \approx 23.0\%$ of crashes
3. State Highway (3), accounting for $205/1067 \approx 19.2\%$ of crashes

Not sure what the "Other" category really entails.

In [ ]:
accident_processed_df.value_counts('RUR_URB')

In [ ]:
sns.countplot(accident_processed_df, x='RUR_URB')

It makes sense that most accidents occur in urban areas, since that's where most people live.

In [ ]:
accident_processed_df.value_counts('FUNC_SYS')

In [ ]:
sns.countplot(accident_processed_df, x='FUNC_SYS')

In order:
1. Principal Arterial - Other (3), accounting for $370/1067 \approx 34.7\%$ of crashes
2. Interstate (1), accounting for $239/1067 \approx 22.4\%$ of crashes
3. Minor Arterial (4), accounting for $204/1067 \approx 19.1\%$ of crashes
4. Principal Arterial - Other Freeways and Expressways (2), accounting for $127/1067 \approx 11.9\%$ of crashes
5. Major Collector (5), accounting for $74/1067 \approx 6.94\%$ of crashes
6. Local (7), accounting for $48/1067 \approx 4.50\%$ of crashes
7. Minor Collector (6), accounting for $5/1067 \approx 0.469\%$ of crashes

For definitions of these terms, see the [Highway Functional Classification Concepts, Criteria, and Procedures](https://web.archive.org/web/20250301002527/https://epa-sdcc.ornl.gov/documents/2023_FHWA_Functional_Classification_Guidelines.pdf) by the Federal Highway Administration (FHWA).

It might be worth looking at [Motor Vehicle Fatalities, Vehicle-Miles, and Associated Rates by Highway Functional System](https://web.archive.org/web/20250222134436/https://www.bts.gov/content/motor-vehicle-fatalities-vehicle-miles-and-associated-rates-highway-functional-system) from the Bureau of Transportation Statistics (BTS).

In [ ]:
accident_processed_df.value_counts('RELJCT1')

In [ ]:
sns.countplot(accident_processed_df, x='RELJCT1')

Evidently, most accidents do not occur within an interchange area.

In [ ]:
accident_processed_df.value_counts('RELJCT2')

In [ ]:
sns.countplot(accident_processed_df, x='RELJCT2')

The top 3 junction types account for $885/1067 \approx 82.9\%$ of crashes.
1. Non-Junction (1), accounting for $586/1067 \approx 54.9\%$ of crashes
2. Intersection-Related (3), accounting for $162/1067 \approx 15.2\%$ of crashes
3. Intersection (2), accounting for $137/1067 \approx 12.8\%$ of crashes

In [ ]:
accident_processed_df.value_counts('TYP_INT')

In [ ]:
sns.countplot(accident_processed_df, x='TYP_INT')

The top 3 intersections for accidents account for $1064/1067 \approx 99.7\%$ of crashes.
1. Not an Intersection (1), accounting for $768/1067 \approx 72.0\%$ of crashes
2. Four-Way Intersection (2), accounting for $203/1067 \approx 19.0\%$ of crashes
3. T-Intersection (3), accounting for $93/1067 \approx 8.72\%$ of crashes
4. Y-Intersection (4), accounting for $5/1067 \approx 0.469\%$ of crashes

Notably, there are zero crashes at Traffic Circles, Roundabouts, Five-Point (or More), L-Intersection, etc.

In [ ]:
accident_processed_df.value_counts('REL_ROAD')

In [ ]:
sns.countplot(accident_processed_df, x='REL_ROAD')

The top 2 positions in the roadway account for $985/1067 \approx 92.3\%$ of crashes.
1. On Roadway (1), accounting for $769/1067 \approx 72.1\%$ of crashes
2. On Roadside (4), accounting for $216/1067 \approx 20.2\%$ of crashes


## Visualizations

Let's try some more visualizations.

Let's focus on a few variables.
- Circular: `MONTH`, `HOUR`, `DAY_WEEK`
- Numerical: `PERSONS`, `PEDS`, `VE_TOTAL`, `PVH_INVL`, `FATALS`
- Nominal: `HARM_EV`, `LGT_COND`, `WEATHER`, `ROUTE`, `RUR_URB`, `FUNC_SYS`, `RELJCT1`, `RELJCT2`, `TYP_INT`, `REL_ROAD`


In [ ]:
relevant_circular_vars = ['MONTH', 'HOUR', 'DAY_WEEK']
relevant_numerical_vars = ['PERSONS', 'PEDS', 'VE_TOTAL', 'PVH_INVL', 'FATALS']
relevant_nominal_vars = ['HARM_EV', 'LGT_COND', 'WEATHER', 'ROUTE', 'RUR_URB', 'FUNC_SYS', 'RELJCT1', 'RELJCT2', 'TYP_INT', 'REL_ROAD']
all_vars = [*relevant_circular_vars, *relevant_numerical_vars, *relevant_nominal_vars]

Let's graph the variables pairwise.

In [ ]:
g = sns.PairGrid(accident_processed_df, vars = all_vars)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend

Uhhh...not sure how helpful that was.

# Investigating EPA data with GeoPandas

I will be using GeoPandas to look through the EPA data.

In [ ]:
import geopandas

In [ ]:
smart_location_df = geopandas.read_file("../data/raw/EPA/SmartLocationDatabaseV3/SmartLocationDatabase.gdb")
smart_location_df

In [ ]:
smart_location_df.info

In [ ]:
smart_location_df.dtypes

The [Technical Documentation and User Guide](https://web.archive.org/web/20250317025004/https://www.epa.gov/system/files/documents/2023-10/epa_sld_3.0_technicaldocumentationuserguide_may2021_0.pdf) describes each column.
Here are a few columns that we might care about.
- `GEOID20` is the census block group 12-digit FIPS code as of 2018. For each car crash in our Kaggle or FARS data, we can use its latitude-longitude data to determine the census block group where the crash occurred.
- `COUNTYFP` is the county FIPS code, which will allow us to select for LA, SF, OC, and SD counties.
- "Core-based statistical area measures"
    - "CSBA" means [core-based statistical area](https://en.wikipedia.org/wiki/List_of_core-based_statistical_areas). [Census block groups](https://en.wikipedia.org/wiki/Census_block_group) (CBGs) are contained in CSBAs.
    - `CBSA_POP` is total population in a CBSA
    - `CBSA_Emp` is total employment in a CBSA
    - `CBSA_Wrk` is total workers living in the CBSA
- Demographics
    - `TotPop` is total population, 2018
    - `CountHU` is total housing units, 2018
    - `HH` is households, 2018
    - `P_WrkAge` is percentage of population that is working age (18-64), 2018
    - `AutoOwn0` is number of households that own zero automobiles, 2018

#TODO

It's great that we can open up the data with GeoPandas, but we may also want to visualize maps.
However, `smart_location_df` has over 200k rows, so for our first demonstration, maybe we should restrict to just LA County.

In [ ]:
los_angeles_df = smart_location_df[(smart_location_df['COUNTYFP'] == "037") & (smart_location_df['STATEFP'] == "06")]
los_angeles_df

In [ ]:
los_angeles_df.plot()

Okay, that looks about right, except we are including Catalina and San Clemente Islands...
Looking them up in the [California Census Map](https://web.archive.org/web/20250313162837/https://census.ca.gov/htc-map/) and the [LA County Census Tract Map](https://web.archive.org/web/20250320001704/https://www2.census.gov/geo/maps/DC2020/PL20/st06_ca/censustract_maps/c06037_los_angeles/DC20CT_C06037.pdf), we find that the two islands mostly correspond to Census Tract 5991, with the town of Avalon corresponding to Census Tract 5990. Let's remove them from the dataframe.

In [ ]:
los_angeles_df = los_angeles_df[(los_angeles_df["TRACTCE"] != "599000") & (los_angeles_df["TRACTCE"] != "599100")]
los_angeles_df.plot()


Nice. Now we can test out some fancier visualizations.

In [ ]:
los_angeles_df.plot("D4A", legend=True) # D4A = distance from population-weighted centroid to nearest transit stop (meters)

In [ ]:
los_angeles_df["D4A"].describe()

In [ ]:
los_angeles_transit_df = los_angeles_df[los_angeles_df["D4A"] != -99999]
los_angeles_transit_df

In [ ]:
los_angeles_transit_df["D4A"].describe()

In [ ]:
los_angeles_transit_df.explore("D4A", legend=True)

In [ ]:
los_angeles_df.explore("D3A", legend=True) # D3A = total road network density

In [ ]:
los_angeles_df.explore("D3APO", legend=True) # D3APO = Network density in terms of facility miles of pedestrian-oriented links per square mile

At this point, I think we're ready to start merging the EPA and Kaggle datasets.